In [1]:
from icepyx import icesat2data as ipd
import os, glob, re, h5py, sys, pyproj
import matplotlib as plt
import shutil
import numpy as np
from pprint import pprint
from astropy.time import Time
from scipy.signal import correlate, detrend
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib widget
import pointCollection as pc

In [2]:
### Where are the data to be processed
#datapath = '/home/jovyan/shared/surface_velocity/FIS_ATL06'

#local data path
datapath = '/media/rag110/ADATA SD700/ICESat2/download/FIS'

ATL06_files=glob.glob(os.path.join(datapath, '*.h5'))


### Where to save the results
#out_path = '/home/jovyan/shared/surface_velocity/ATL06_out/'


#local out_path

out_path = '/media/rag110/ADATA SD700/ICESat2/output/FIS/'


In [ ]:
import numpy as np
import pointCollection as pc
#old copy of of rodrigo function
# def add_surface_velocity_to_is2_dict(is2_dict, spatial_extent, path, vel_x, vel_y ):
#     """
    
#     is2_dict: Python dictionary with ATL06 track data
#     spatial_extent: bounding box of the interest area in the format:
#                     (e.g. [-65, -86, -55, -81] == [min_lon, min_lat, max_lon, max_lat])
#     path: local path to velocity data
#     vel_x: tif velocity raster with x component
#     vel_y: tif velocity raster with y component
    
#     """
#     data_root = path
    
#     spatial_extent = np.array([spatial_extent])
#     lat=spatial_extent[[1, 3, 3, 1, 1]]
#     lon=spatial_extent[[2, 2, 0, 0, 2]]
#     print(lat)
#     print(lon)
#     # project the coordinates to Antarctic polar stereographic
#     xy=np.array(pyproj.Proj(3031)(lon, lat))
#     # get the bounds of the projected coordinates 
#     XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
#     YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
    
#     #Measures_vx=pc.grid.data().from_geotif(os.path.join(data_root,vel_x), bounds=[XR, YR])
#     #Measures_vy=pc.grid.data().from_geotif(os.path.join(data_root,vel_y), bounds=[XR, YR])
    
#     Measures_vx=pc.grid.data().from_geotif(os.path.join(data_root,vel_x), bounds=[XR, YR])
#     Measures_vy=pc.grid.data().from_geotif(os.path.join(data_root,vel_y), bounds=[XR, YR])
    
#     vx = Measures_vx.interp(is2_dict['x'],is2_dict['y'])
#     vy = Measures_vy.interp(is2_dict['x'],is2_dict['y'])

#     #Solve for angle to rotate Vy to be along track and Vx to be across track
#     import math
#     xL=abs((is2_dict['x'][0])-(is2_dict['x'][1]))
#     yL=abs((is2_dict['y'][0])-(is2_dict['y'][1]))

#     #decides if is descending or ascending path
#     if is2_dict['x'][0]-is2_dict['x'][1] < 0:

#         theta_rad=math.atan(xL/yL)
#         #theta_deg=theta_rad*180/math.pi
#         is2_dict['v_along']=vy/math.cos(theta_rad)
#         is2_dict['v_across']=vx/math.cos(theta_rad)

#     else:
    
#         theta_rad=math.atan(xL/yL)
#         #theta_deg=theta_rad*180/math.pi
#         is2_dict['v_along']=vy/math.sin(theta_rad)
#         is2_dict['v_across']=vx/math.sin(theta_rad)
    
#     #is2_dict['Vdiff']=vy-v_along
#     return is2_dict 


def add_surface_velocity_to_is2_dict(x_ps_beam, y_ps_beam , spatial_extent, vel_x_path, vel_y_path):
    """
    
    is2_dict: Python dictionary with ATL06 track data
    spatial_extent: bounding box of the interest area in the format:
                    (e.g. [-65, -86, -55, -81] == [min_lon, min_lat, max_lon, max_lat])
    path: local path to velocity data
    vel_x: tif velocity raster with x component
    vel_y: tif velocity raster with y component
    
    """
    
    #fix with if statement about type of list or array DONE
    
    if type(spatial_extent) == type([]):
    
        spatial_extent = np.array([spatial_extent])
    
        
    lat=spatial_extent[[1, 3, 3, 1, 1]]
    lon=spatial_extent[[2, 2, 0, 0, 2]]

    # project the coordinates to Antarctic polar stereographic
    xy=np.array(pyproj.Proj(3031)(lon, lat))
    # get the bounds of the projected coordinates 
    XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
    YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
    
    #Measures_vx=pc.grid.data().from_geotif(os.path.join(data_root,vel_x), bounds=[XR, YR])
    #Measures_vy=pc.grid.data().from_geotif(os.path.join(data_root,vel_y), bounds=[XR, YR])
    
    Measures_vx=pc.grid.data().from_geotif(vel_x_path, bounds=[XR, YR])
    Measures_vy=pc.grid.data().from_geotif(vel_y_path, bounds=[XR, YR])
    
    vx = Measures_vx.interp(x_ps_beam,y_ps_beam)
    vy = Measures_vy.interp(x_ps_beam,y_ps_beam)

    #Solve for angle to rotate Vy to be along track and Vx to be across track
    import math
    xL=abs((x_ps_beam[0])-(x_ps_beam[1]))
    yL=abs((y_ps_beam[0])-(y_ps_beam[1]))

    #decides if is descending or ascending path
    if x_ps_beam[0]-x_ps_beam[1] < 0:

        theta_rad=math.atan(xL/yL)
        #theta_deg=theta_rad*180/math.pi
        v_along=vy/math.cos(theta_rad)
        #v_across=vx/math.cos(theta_rad)

    else:
    
        theta_rad=math.atan(xL/yL)
        #theta_deg=theta_rad*180/math.pi
        v_along=vy/math.sin(theta_rad)
        #v_across=vx/math.sin(theta_rad)
    
    #Vdiff=vy-v_along
    return v_along

# Get a list of all available repeat ground tracks in the folder with data in it:

In [11]:
rgts = {}
for filepath in ATL06_files:
    filename = filepath.split('/')[-1]
    rgt = filename.split('_')[3][0:4]
    track = filename.split('_')[3][4:6]
#     print(rgt,track)
    if not rgt in rgts.keys():
        rgts[rgt] = []
        rgts[rgt].append(track)
    else:
        rgts[rgt].append(track)


# all rgt values in our study are are in rgts.keys()
print(rgts.keys())

# available tracks for each rgt are in rgts[rgt]; ex.:
print(rgts['0848'])




dict_keys(['0711', '0720', '0726', '0735', '0741', '0750', '0751', '0756', '0766', '0772', '0787', '0796', '0802', '0811', '0812', '0817', '0827', '0832', '0833', '0842', '0857', '0863', '0872', '0873', '0878', '0888', '0893', '0894', '0903', '0909', '0781', '0848', '0918', '0994', '1070', '1132', '1198', '1275', '1345', '0025', '0924', '0933', '0939', '0949', '0954', '0955', '0964', '0970', '0979', '0985', '1000', '1010', '1015', '1016', '1025', '1031', '1040', '1046', '1055', '1061', '1071', '1076', '1077', '1086', '1092', '1101', '1107', '1116', '1122', '1131', '1137', '1138', '1147', '1153', '1162', '1168', '1177', '1183', '1192', '1193', '1208', '1214', '1223', '1229', '1238', '1244', '1253', '1254', '1259', '1269', '1284', '1290', '1299', '1305', '1314', '1315', '1320', '1330', '1335', '1336', '1351', '1360', '1366', '1375', '1376', '1381', '0004', '0009', '0010', '0019', '0040', '0049', '0055', '0065', '0070', '0071', '0080', '0086', '0101', '0110', '0126', '0131', '0132', '0141

In [12]:
### Revised version of code from Ben Smith to read in the hdf5 files and extract necessary datasets and information
def atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=None):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            beam: a string specifying which beam is to be read (ex: gt1l, gt1r, gt2l, etc)
            field_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
            index: which entries in each field to read
            epsg: an EPSG code specifying a projection (see www.epsg.org).  Good choices are:
                for Greenland, 3413 (polar stereographic projection, with Greenland along the Y axis)
                for Antarctica, 3031 (polar stereographic projection, centered on the Pouth Pole)
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a numpy array containing the 
                data
    """
    if field_dict is None:
        field_dict={None:['latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}
    D={}
    # below: file_re = regular expression, it will pull apart the regular expression to get the information from the filename
    file_re=re.compile('ATL06_(?P<date>\d+)_(?P<rgt>\d\d\d\d)(?P<cycle>\d\d)(?P<region>\d\d)_(?P<release>\d\d\d)_(?P<version>\d\d).h5')
    with h5py.File(filename,'r') as h5f:
        for key in field_dict:
            for ds in field_dict[key]:
                if key is not None:
                    ds_name=beam+'/land_ice_segments/'+key+'/'+ds
                else:
                    ds_name=beam+'/land_ice_segments/'+ds
                if index is not None:
                    D[ds]=np.array(h5f[ds_name][index])
                else:
                    D[ds]=np.array(h5f[ds_name])
                if '_FillValue' in h5f[ds_name].attrs:
                    bad_vals=D[ds]==h5f[ds_name].attrs['_FillValue']
                    D[ds]=D[ds].astype(float)
                    D[ds][bad_vals]=np.NaN
        D['data_start_utc'] = h5f['/ancillary_data/data_start_utc'][:]
        D['delta_time'] = h5f['/' + beam + '/land_ice_segments/delta_time'][:]
        D['segment_id'] = h5f['/' + beam + '/land_ice_segments/segment_id'][:]
    if epsg is not None:
        xy=np.array(pyproj.proj.Proj(epsg)(D['longitude'], D['latitude']))
        D['x']=xy[0,:].reshape(D['latitude'].shape)
        D['y']=xy[1,:].reshape(D['latitude'].shape)
    temp=file_re.search(filename)
    D['rgt']=int(temp['rgt'])
    D['cycle']=int(temp['cycle'])
    D['beam']=beam
    return D

# Loop over available rgts and do the correlation processing:

Save all results as hdf5 files that can be reloaded and manipulated laver

In [13]:
### Some functions
# MISSING HERE: mask by data quality?
def load_data_by_rgt(rgt, smoothing, smoothing_window_size, dx, path_to_data, product):
    """ 
    rgt: repeat ground track number of desired data
    smoothing: if true, a centered running avergae filter of smoothing_window_size will be used
    smoothing_window_size: how large a smoothing window to use (in meters)
    dx: desired spacing 
    path_to_data: 
    product: ex., ATL06
    """ 
    
    # hard code these for now:
    cycles = ['03','04','05','06','07'] # not doing 1 and 2, because don't overlap exactly
    beams = ['gt1l','gt1r','gt2l','gt2r','gt3l','gt3r'] 

    ### extract data from all available cycles
    x_atc = {}
    lats = {}
    lons = {}
    h_li_raw = {} # unsmoothed data; equally spaced x_atc, still has nans 
    h_li_raw_NoNans = {} # unsmoothed data; equally spaced x_atc, nans filled with noise
    h_li = {} # smoothed data, equally spaced x_atc, nans filled with noise 
    h_li_diff = {}
    times = {}
    min_seg_ids = {}
    segment_ids = {}
    x_ps= {}
    y_ps= {}

    cycles_this_rgt = []
    for cycle in cycles: # loop over all available cycles
        Di = {}
        x_atc[cycle] = {}
        lats[cycle] = {}
        lons[cycle] = {}
        h_li_raw[cycle] = {}
        h_li_raw_NoNans[cycle] = {}
        h_li[cycle] = {}
        h_li_diff[cycle] = {}
        times[cycle] = {}
        min_seg_ids[cycle] = {}
        segment_ids[cycle] = {}
        x_ps[cycle]= {}
        y_ps[cycle]= {}


        filenames = glob.glob(os.path.join(path_to_data, f'*{product}_*_{rgt}{cycle}*_003*.h5'))
        error_count=0


        for filename in filenames: # try and load any available files; hopefully is just one
            try:
                for beam in beams:
                    Di[filename]=atl06_to_dict(filename,'/'+ beam, index=None, epsg=3031)
                    

                    
                    times[cycle][beam] = Di[filename]['data_start_utc']

                    # extract h_li and x_atc, and lat/lons for that section                
                    x_atc_tmp = Di[filename]['x_atc']
                    h_li_tmp = Di[filename]['h_li']#[ixs]
                    lats_tmp = Di[filename]['latitude']
                    lons_tmp = Di[filename]['longitude']
                    x_ps_tmp = Di[filename]['x']
                    y_ps_tmp= Di[filename]['y']


                    # segment ids:
                    seg_ids = Di[filename]['segment_id']
                    min_seg_ids[cycle][beam] = seg_ids[0]
                    #print(len(seg_ids), len(x_atc_tmp))

                    # make a monotonically increasing x vector
                    # assumes dx = 20 exactly, so be carefull referencing back
                    ind = seg_ids - np.nanmin(seg_ids) # indices starting at zero, using the segment_id field, so any skipped segment will be kept in correct location
                    x_full = np.arange(np.max(ind)+1) * 20 + x_atc_tmp[0]
                    h_full = np.zeros(np.max(ind)+1) + np.NaN
                    h_full[ind] = h_li_tmp
                    lats_full = np.zeros(np.shape(x_full)) * np.nan
                    lats_full[ind] = lats_tmp
                    lons_full = np.zeros(np.shape(x_full)) * np.nan
                    lons_full[ind] = lons_tmp
                    x_ps_full = np.zeros(np.shape(x_full)) * np.nan
                    x_ps_full[ind] = x_ps_tmp
                    y_ps_full = np.zeros(np.shape(x_full)) * np.nan
                    y_ps_full[ind] = y_ps_tmp

                    ## save the segment id's themselves, with gaps filled in
                    segment_ids[cycle][beam] = np.zeros(np.max(ind)+1) + np.NaN
                    segment_ids[cycle][beam][ind] = seg_ids


                    x_atc[cycle][beam] = x_full
                    h_li_raw[cycle][beam] = h_full # preserves nan values
                    lons[cycle][beam] = lons_full
                    lats[cycle][beam] = lats_full
                    x_ps[cycle][beam] = x_ps_full
                    y_ps[cycle][beam] = y_ps_full

                    ### fill in nans with noise h_li datasets
            #                         h = ma.array(h_full,mask =np.isnan(h_full)) # created a masked array, mask is where the nans are
            #                         h_full_filled = h.mask * (np.random.randn(*h.shape)) # fill in all the nans with random noise

                    ### interpolate nans in pandas
                    # put in dataframe for just this step; eventually rewrite to use only dataframes?              
                    data = {'x_full': x_full, 'h_full': h_full}
                    df = pd.DataFrame(data, columns = ['x_full','h_full'])
                    #df.plot(x='x_full',y='h_full')
                    # linear interpolation for now
                    df['h_full'].interpolate(method = 'linear', inplace = True)
                    h_full_interp = df['h_full'].values
                    h_li_raw_NoNans[cycle][beam] = h_full_interp # has filled nan values


                    # running average smoother /filter
                    if smoothing == True:
                        h_smoothed = (1/smoothing_window_size) * np.convolve(filt, h_full_interp, mode = 'same')
                        h_li[cycle][beam] = h_smoothed

                        # differentiate that section of data
                        h_diff = (h_smoothed[1:] - h_smoothed[0:-1]) / (x_full[1:] - x_full[0:-1])
                    else: 
                        h_li[cycle][beam] = h_full_interp
                        h_diff = (h_full_interp[1:] - h_full_interp[0:-1]) / (x_full[1:] - x_full[0:-1])
                    h_li_diff[cycle][beam] = h_diff



                    #print(len(x_full), len(h_full), len(lats_full), len(seg_ids), len(h_full_interp), len(h_diff))


                cycles_this_rgt+=[cycle]
            except KeyError as e:
                print(f'file {filename} encountered error {e}')
                error_count += 1

    print('Cycles available: ' + ','.join(cycles_this_rgt))
    return x_atc, lats, lons, h_li_raw, h_li_raw_NoNans, h_li, h_li_diff, \
            times, min_seg_ids, segment_ids, cycles_this_rgt, x_ps, y_ps
    
    

Choices about processing:

In [23]:
### Which cycles to process
cycles = ['03','04','05','06','07'] # not doing 1 and 2, because don't overlap exactly (this could be future work)

### Which beams to process
beams = ['gt1l','gt1r','gt2l','gt2r','gt3l','gt3r']

### Which product
product = 'ATL06'
dx = 20 # x_atc coordinate distance, will be different for different products or processed data

# Filter preprocessing: 
smoothing = True # Whether or not to apply a running average filter
smoothing_window_size = int(np.round(60 / dx)) # meters / dx [meters]; this is the number of datapoints to smooth over
# ex., 60 m smoothing window is a 3 point running average smoothed dataset if dx = 20 for ATL06
filt = np.ones(smoothing_window_size) # create the filter to convolve with the data

### Control the correlation step:
segment_length = 3000 # meters, how wide is the window we are correlating in each step
search_width = 800 # meters, how far in front of and behind the window to check for correlation
along_track_step = 100 # meters; how much to jump between each consecutivevelocity determination
max_percent_nans = 10 # Maximum % of segment length that can be nans and still do the correlation step

#Measures spatial extent and paths to tif files
spatial_extent = np.array([-65, -86, -55, -81])

#Rodrigo Gomez Fell computer path @ UC
measures_Vx_path = '/mnt/user1/Antarctica/Quantarctica3/Glaciology/MEaSUREs Ice Flow Velocity/anta_phase_map_VX.tif'
measures_Vy_path = '/mnt/user1/Antarctica/Quantarctica3/Glaciology/MEaSUREs Ice Flow Velocity/anta_phase_map_VY.tif'




Do the correlation processing:

In [102]:
### Create dictionaries to put info in
velocities = {}   
correlations = {}     
lags = {}
x_atcs_for_velocities = {}
latitudes = {}
longitudes = {}

#add a v_along variable to test if the problem of the length of measures_v_along is the loop


rgts_with_errors = []
total_number_repeat_tracks_processed = 0

### Loop over each rgt in the data directory
for ir, rgt in enumerate(rgts.keys()):
    if ir < len(rgts.keys()):  # this is here in case you want to look at specific rgts
        try:
            print('\nProcessing rgt ' + rgt + ', #' +str(ir) + ' of ' + str(len(rgts.keys())))

            ### Determine how many files there are for this rgt
            rgt_files = glob.glob(os.path.join(datapath, f'*ATL06_*_{rgt}*_003*.h5'))
            n_rgt_files_cycle3_and_after = 0
            for file in rgt_files:
                if float(file.split('/')[-1].split('_')[3][4:6]) >= 3:
                    n_rgt_files_cycle3_and_after += 1

            print('There are ' +str(n_rgt_files_cycle3_and_after) + ' files available for this track from cycle 3 onward')


            ### Only process if there is at least one repeat track during the time period when data overlapped
            if n_rgt_files_cycle3_and_after >= 2:


                ### Load necessary data from all available cycles
                x_atc, lats, lons, h_li_raw, h_li_raw_NoNans, h_li, h_li_diff, times, min_seg_ids, segment_ids, cycles_this_rgt, x_ps, y_ps = \
                    load_data_by_rgt(rgt, smoothing, smoothing_window_size, dx, datapath, product)
                
                ### Determine # of possible velocities, given how many cycles are available:
                n_possible_veloc = len(cycles_this_rgt) -1 # naive, for now; can improve later; We could, for example, do non-consecutive cycles, like 03 and 05
                for veloc_number in range(n_possible_veloc):
                    
                    ### Where to save the results:
                    h5_file_out = f'{out_path}rgt{rgt}_veloc{veloc_number}.hdf5'
                    
                    ### Save some metadata
                    with h5py.File(h5_file_out,'w') as f:
                        f['dx'] = dx 
                        f['product'] = product 
                        f['segment_length'] = segment_length 
                        f['search_width'] = search_width 
                        f['along_track_step'] = along_track_step 
                        f['max_percent_nans'] = max_percent_nans 
                        f['smoothing'] = smoothing 
                        f['smoothing_window_size'] = smoothing_window_size 
                        f['process_date'] = str(Time.now().value) 


                    ### Which cycles are being processed in the current velocity determination
                    cycle1 = cycles_this_rgt[veloc_number]
                    cycle2 = cycles_this_rgt[veloc_number+1]
                    
                    ### Timing of each cycle in the current velocity determination
                    t1_string = times[cycle1]['gt1l'][0].astype(str) #figure out later if just picking hte first one it ok
                    t1 = Time(t1_string)

                    t2_string = times[cycle2]['gt1l'][0].astype(str) #figure out later if just picking hte first one it ok
                    t2 = Time(t2_string)

                    ### Elapsed time between cycles
                    dt = (t2 - t1).jd # difference in julian days

                    ### Create dictionaries
                    velocities[rgt] = {}   
                    correlations[rgt] = {}     
                    lags[rgt] = {}

                    ### Loop over each beam
                    for beam in beams:

                        ### Determine x1s, which are the x_atc coordinates at which each cut out window begins
                        # To be common between both repeats, the first point x1 needs to be the larger first value between repeats
                        min_x_atc_cycle1 = x_atc[cycle1][beam][0]
                        min_x_atc_cycle2 = x_atc[cycle2][beam][0]

                        # pick out the track that starts at greater x_atc, and use that as x1s vector
                        if min_x_atc_cycle1 != min_x_atc_cycle2: 
                            x1 = np.nanmax([min_x_atc_cycle1,min_x_atc_cycle2])
                            cycle_n = np.arange(0,2)[[min_x_atc_cycle1,min_x_atc_cycle2] == x1][0]
                            if cycle_n == 0:
                                cycletmp = cycle2
                            elif cycle_n == 1:
                                cycletmp = cycle1
                            n_segments_this_track = (len(x_atc[cycletmp][beam]) - search_width/dx) / (along_track_step/dx)
                            
                            ### Generate the x1s vector, in the case that the repeat tracks don't start in the same place
                            x1s = x_atc[cycletmp][beam][int(search_width/dx)+1::int(search_width/dx)]
                            # start at search_width/dx in, so the code never tries to get data outside the edges of this rgt
                            # add 1 bc the data are differentiated, and h_li_diff is therefore one point shorter

                        elif min_x_atc_cycle1 == min_x_atc_cycle2: # doesn't matter which cycle
                            ### Generate the x1s vector, in the case that the repeat tracks do start in the same place
                            x1s = x_atc[cycle1][beam][int(search_width/dx)+1::int(search_width/dx)]

                        ### Determine xend, where the x1s vector ends: smaller value for both beams, if different
                        max_x_atc_cycle1 = x_atc[cycle1][beam][-1]
                        max_x_atc_cycle2 = x_atc[cycle2][beam][-1]
                        smallest_xatc = np.min([max_x_atc_cycle1,max_x_atc_cycle2])
                        ixmax = np.where(x1s >= smallest_xatc - search_width/dx)
                        if len(ixmax[0]) >= 1:
                            ixtmp = ixmax[0][0]
                            x1s = x1s[:ixtmp]

                        ### Create vectors to store results in
                        velocities[rgt][beam] = np.empty_like(x1s)
                        correlations[rgt][beam] = np.empty_like(x1s)
                        lags[rgt][beam] = np.empty_like(x1s)

                        midpoints_x_atc = np.empty(np.shape(x1s)) # for writing out 
                        midpoints_lat = np.empty(np.shape(x1s)) # for writing out 
                        midpoints_lon = np.empty(np.shape(x1s)) # for writing out 
                        midpoints_seg_ids = np.empty(np.shape(x1s)) # for writing out 
                                                                                
                        ### Entire x_atc vectors for both cycles    
                        x_full_t1 = x_atc[cycle1][beam]
                        x_full_t2 = x_atc[cycle2][beam]

                        ### Loop over x1s, positions along track that each window starts at
                        for xi, x1 in enumerate(x1s):
                            
                            ### Cut out data: small chunk of data at time t1 (first cycle)
                            ix_x1 = np.arange(len(x_full_t1))[x_full_t1 >= x1][0] # Index of first point that is greater than x1
                            ix_x2 = ix_x1 + int(np.round(segment_length/dx)) # ix_x1 + number of datapoints within the desired segment length
                            x_t1 = x_full_t1[ix_x1:ix_x2] # cut out x_atc values, first cycle
                            lats_t1 = lats[cycle1][beam][ix_x1:ix_x2] # cut out latitude values, first cycle
                            lons_t1 = lons[cycle1][beam][ix_x1:ix_x2] # cut out longitude values, first cycle
                            seg_ids_t1 = segment_ids[cycle1][beam][ix_x1:ix_x2] # cut out segment_ids, first cycle
                            h_li1 = h_li_diff[cycle1][beam][ix_x1-1:ix_x2-1] # cut out land ice height values, first cycle; start 1 index earlier because 
                            # the h_li_diff data are differentiated, and therefore one sample shorter

                            # Find segment midpoints; this is the position where we will assign the velocity measurement from each window
                            n = len(x_t1)
                            midpt_ix = int(np.floor(n/2))
                            midpoints_x_atc[xi] = x_t1[midpt_ix]
                            midpoints_lat[xi] = lats_t1[midpt_ix]
                            midpoints_lon[xi] = lons_t1[midpt_ix]
                            midpoints_seg_ids[xi] = seg_ids_t1[midpt_ix]
                            
                            ### Cut out data: wider chunk of data at time t2 (second cycle)
                            ix_x3 = ix_x1 - int(np.round(search_width/dx)) # extend on earlier end by number of indices in search_width
                            ix_x4 = ix_x2 + int(np.round(search_width/dx)) # extend on later end by number of indices in search_width
                            x_t2 = x_full_t2[ix_x3:ix_x4] # cut out x_atc values, second cycle
                            h_li2 = h_li_diff[cycle2][beam][ix_x3-1:ix_x4-1]# cut out land ice height values, second cycle; start 1 index earlier because 
                            # the h_li_diff data are differentiated, and therefore one sample shorter

                            ### Determine number of nans in each data chunk
                            n_nans1 = np.sum(np.isnan(h_li_raw[cycle1][beam][ix_x1:ix_x2]))
                            n_nans2 = np.sum(np.isnan(h_li_raw[cycle2][beam][ix_x3:ix_x4]))
                            
                            ### Only process if there are fewer than 10% nans in either data chunk:
                            if (n_nans1 / len(h_li1) <= max_percent_nans/100) and (n_nans2 / len(h_li2) <= max_percent_nans/100):

                                # Detrend both chunks of data
                                h_li1 = detrend(h_li1,type = 'linear')
                                h_li2 = detrend(h_li2,type = 'linear')

                                # Normalize both chunks of data, if desired
                                # h_li1 = h_li1 / np.nanmax(np.abs(h_li1))
                                # h_li2 = h_li2 / np.nanmax(np.abs(h_li2))

                                ### Correlate the old and new data
                                # We made the second data vector longer than the first, so the valid method returns values
                                corr = correlate(h_li1, h_li2, mode = 'valid', method = 'direct') 

                                ### Normalize correlation function by autocorrelations
                                # Normalizing coefficient changes with each step along track; this section determines a changing along track normalizing coefficiant
                                coeff_a_val = np.sum(h_li1**2)
                                coeff_b_val = np.zeros(len(h_li2) - len(h_li1)+1)
                                for shift in range(len(h_li2) - len(h_li1)+1):
                                    h_li2_section = h_li2[shift:shift + len(h_li1)]
                                    coeff_b_val[shift] = np.sum(h_li2_section **2)
                                norm_vec = np.sqrt(coeff_a_val * coeff_b_val)
                                corr_normed = corr / np.flip(norm_vec) # i don't really understand why this has to flip, but otherwise it yields correlation values above 1...

                                ### Create a vector of lags for the correlation function
                                lagvec = np.arange(- int(np.round(search_width/dx)), int(search_width/dx) +1,1)# for mode = 'valid'

                                ### Convert lag to distance
                                shift_vec = lagvec * dx

                                ### ID peak correlation coefficient
                                ix_peak = np.arange(len(corr_normed))[corr_normed == np.nanmax(corr_normed)][0]
                                
                                ### Save correlation coefficient, best lag, velocity, etc at the location of peak correlation coefficient
                                best_lag = lagvec[ix_peak]
                                best_shift = shift_vec[ix_peak]
                                velocities[rgt][beam][xi] = best_shift/(dt/365)
                                correlations[rgt][beam][xi] = corr_normed[ix_peak]
                                lags[rgt][beam][xi] = lagvec[ix_peak]
                            else:
                                ### If there are too many nans, just save a nan
                                velocities[rgt][beam][xi] = np.nan
                                correlations[rgt][beam][xi] = np.nan
                                lags[rgt][beam][xi] = np.nan
                                
                         
                        xy=np.array(pyproj.Proj(3031)(midpoints_lon, midpoints_lat))    
                        ### Add velocities to hdf5 file for each beam
                        with h5py.File(h5_file_out, 'a') as f:
                            f[beam +'/x_atc'] = midpoints_x_atc # assign x_atc value of half way along the segment
                            f[beam +'/latitudes'] = midpoints_lat # assign x_atc value of half way along the segment
                            f[beam +'/longitudes'] = midpoints_lon # assign x_atc value of half way along the segment
                            f[beam +'/velocities'] = velocities[rgt][beam] # assign x_atc value of half way along the segment
                            f[beam +'/correlation_coefficients'] = correlations[rgt][beam] # assign x_atc value of half way along the segment
                            f[beam +'/best_lags'] = lags[rgt][beam] # assign x_atc value of half way along the segment
                            f[beam +'/segment_ids'] = midpoints_seg_ids
                            f[beam +'/first_cycle_time'] = str(Time(times[cycle1][beam][0]))
                            f[beam +'/second_cycle_time'] = str(Time(times[cycle2][beam][0]))
                            f[beam +'/Measures_v_along'] = add_surface_velocity_to_is2_dict(xy[0], xy[1] , spatial_extent, measures_Vx_path, measures_Vy_path)


                    ### Record which cycles contributed to these results
                    with h5py.File(h5_file_out, 'a') as f:
                        f['contributing_cycles'] = ','.join([cycle1,cycle2])

            
                total_number_repeat_tracks_processed += 1
                

        except (ValueError, IndexError) as e:
            print(f'rgt {rgt} encountered an error')
            print(e)
            rgts_with_errors.append(rgt)
            
print(f'Total number of repeat tracks successfully processed = {total_number_repeat_tracks_processed}')






Processing rgt 0711, #0 of 220
There are 3 files available for this track from cycle 3 onward
Cycles available: 03,04,05
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]

Processing rgt 0720, #1 of 220
There are 3 files available for this track from cycle 3 onward
Cycles available: 03,04,05
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81

/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/ipykernel_launcher.py:161: RuntimeWarning: divide by zero encountered in long_scalars


rgt 0726 encountered an error
index 137 is out of bounds for axis 0 with size 81

Processing rgt 0735, #3 of 220
There are 3 files available for this track from cycle 3 onward
Cycles available: 03,04,05


/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/ipykernel_launcher.py:161: RuntimeWarning: divide by zero encountered in long_scalars
/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/ipykernel_launcher.py:161: RuntimeWarning: divide by zero encountered in long_scalars


[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]


...

/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/ipykernel_launcher.py:161: RuntimeWarning: divide by zero encountered in long_scalars


[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]


/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/ipykernel_launcher.py:161: RuntimeWarning: divide by zero encountered in long_scalars


[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]

Processing rgt 1071, #60 of 220
There are 3 files available for this track from cycle 3 onward
rgt 1071 encountered an error
operands could not be broadcast together with shapes (2,) (0,) 

Processing rgt 1076, #61 of 220
There are 3 files available for this track from cycle 3 onward
Cycles available: 03,04,05
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]
[-86 -81 -81 -86 -86]
[-55 

KeyboardInterrupt: 


# Load data, make a map of correlation coefficient

In [37]:
### What's in each results file

#!h5ls -r /home/jovyan/shared/surface_velocity/ATL06_out/rgt0589_veloc0.hdf5

!h5ls -r /media/rag110/ADATA\ SD700/ICESat2/output/FIS/rgt0004_veloc0.hdf5

/                        Group
/along_track_step        Dataset {SCALAR}
/contributing_cycles     Dataset {SCALAR}
/dx                      Dataset {SCALAR}
/gt1l                    Group
/gt1l/Measures_v_along   Dataset {5100}
/gt1l/best_lags          Dataset {127}
/gt1l/correlation_coefficients Dataset {127}
/gt1l/first_cycle_time   Dataset {SCALAR}
/gt1l/latitudes          Dataset {127}
/gt1l/longitudes         Dataset {127}
/gt1l/second_cycle_time  Dataset {SCALAR}
/gt1l/segment_ids        Dataset {127}
/gt1l/velocities         Dataset {127}
/gt1l/x_atc              Dataset {127}
/gt1r                    Group
/gt1r/Measures_v_along   Dataset {5112}
/gt1r/best_lags          Dataset {127}
/gt1r/correlation_coefficients Dataset {127}
/gt1r/first_cycle_time   Dataset {SCALAR}
/gt1r/latitudes          Dataset {127}
/gt1r/longitudes         Dataset {127}
/gt1r/second_cycle_time  Dataset {SCALAR}
/gt1r/segment_ids        Dataset {127}
/gt1r/velocities         Dataset {127}
/gt1r/x_atc   

In [51]:
file='/media/rag110/ADATA SD700/ICESat2/output/FIS/rgt0004_veloc0.hdf5'

glob.glob(file)

plt.style.use('seaborn-whitegrid')
fig = plt.figure(figsize=[8,8])

hax0=fig.add_subplot(211)
hax1=fig.add_subplot(212)

hax0.set_title('velocs ' )
hax1.set_title('measures ' )


with h5py.File(file, 'r') as f:
        for beam in beams:
            lats = f[f'/{beam}/latitudes'][()]
            lons = f[f'/{beam}/longitudes'][()]
            coeffs = f[f'/{beam}/correlation_coefficients'][()]
            velocs = f[f'/{beam}/velocities'][()]
            v_along=f[f'/{beam}/Measures_v_along'][()]
            xy=np.array(pyproj.proj.Proj(3031)(lons,lats))

            h0 = hax0.scatter(xy[0], velocs)
            h1 = hax1.scatter(xy[0], v_along)
          
            



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: x and y must be the same size

In [54]:
file='/media/rag110/ADATA SD700/ICESat2/output/FIS/rgt0004_veloc0.hdf5'

velocs=[]
v_along=[]
with h5py.File(file, 'r') as f:
        for beam in beams:
            #lats = f[f'/{beam}/latitudes'][()]
            #lons = f[f'/{beam}/longitudes'][()]
            #coeffs = f[f'/{beam}/correlation_coefficients'][()]
            velocs.append(f[f'/{beam}/velocities'][()])
            v_along.append(f[f'/{beam}/Measures_v_along'][()])
            #xy=np.array(pyproj.proj.Proj(3031)(lons,lats))

            #h0 = hax0.scatter(xy[0], velocs)
            #h1 = hax1.scatter(xy[0], v_along)
          

In [48]:
### MOA parameters
moa_datapath = '/mnt/user1/Antarctica/Quantarctica3/SatelliteImagery/MODIS/'
spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-65, -86, -55, -81])

lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
#MOA=pc.grid.data().from_geotif(os.path.join(moa_datapath, 'MOA','moa_2009_1km.tif'), bounds=[XR, YR])
MOA=pc.grid.data().from_geotif(os.path.join(moa_datapath,'MODIS_Mosaic.tif'), bounds=[XR, YR])

epsg=3031 #PS?

# Plot MOA with correlation coefficient on top
plt.close('all')
plt.figure(figsize=[8,8])
hax0=plt.gcf().add_subplot(111, aspect='equal')
#MOA.show(ax=hax0,cmap='gray', clim=[14000, 17000])
MOA.show(ax=hax0, clim=[14000, 17000])
plt.title('Correlation Coefficient')

results_files = glob.glob(out_path + '/*.hdf5')
for file in results_files:
    with h5py.File(file, 'r') as f:
        for beam in beams:
            try:
                lats = f[f'/{beam}/latitudes'][()]
                lons = f[f'/{beam}/longitudes'][()]
                coeffs = f[f'/{beam}/correlation_coefficients'][()]
                xy=np.array(pyproj.proj.Proj(epsg)(lons,lats))

                h = hax0.scatter(xy[0], xy[1], 0.25, coeffs, vmin = 0, vmax = 1)

            except:
                pass
c = plt.colorbar(h)
c.set_label('Correlation coefficient (0->1)')
outfile = out_path + 'correlation_coefficient.png'
plt.savefig(outfile)


# Plot MOA with best velocity on top
plt.figure(figsize=[8,8])
hax2=plt.gcf().add_subplot(111, aspect='equal')
MOA.show(ax=hax2,cmap='gray', clim=[14000, 17000])
plt.title('Best lag')

results_files = glob.glob(out_path + '/*.hdf5')
for file in results_files:
    with h5py.File(file, 'r') as f:
        for beam in beams:
            try:
                lats = f[f'/{beam}/latitudes'][()]
                lons = f[f'/{beam}/longitudes'][()]
                lags = f[f'/{beam}/best_lags'][()]
                xy=np.array(pyproj.proj.Proj(epsg)(lons,lats))

                h = hax2.scatter(xy[0], xy[1], 0.25, lags, vmin = -10, vmax = 10,cmap='RdBu')

            except:
                pass
c = plt.colorbar(h)
c.set_label(f'Best lag (dx) ={dx}')


outfile = out_path + 'best_lag.png'
plt.savefig(outfile)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'clim': [14000, 17000], 'extent': array([-887950., -356200.,  183825.,  561950.]), 'origin': 'lower'}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356200.,  183825.,  561950.]), 'origin': 'lower'}




# Plot results, masked by correlation coefficient

In [49]:
### Select a correlation threshold
correlation_threshold = 0.65

### MOA parameters
#moa_datapath = '/srv/tutorial-data/land_ice_applications/'
moa_datapath = '/mnt/user1/Antarctica/Quantarctica3/SatelliteImagery/MODIS/'

spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-65, -86, -55, -81])


lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
MOA=pc.grid.data().from_geotif(os.path.join(moa_datapath,'MODIS_Mosaic.tif'), bounds=[XR, YR])

epsg=3031

# Plot MOA with correlation coefficient and velocity on top
plt.close('all')
fig = plt.figure(figsize=[8,8])
hax0=fig.add_subplot(211, aspect='equal')
MOA.show(ax=hax0,cmap='gray', clim=[14000, 17000])
hax1=fig.add_subplot(212, aspect='equal')
MOA.show(ax=hax1,cmap='gray', clim=[14000, 17000])

hax0.set_title('Correlation Coefficient, above correlation threshold ' + str(correlation_threshold))
hax1.set_title('Best velocity, above correlation threshold ' + str(correlation_threshold))

### Loop over results, load data, plot
results_files = glob.glob(out_path + '/*.hdf5')
for file in results_files:
    with h5py.File(file, 'r') as f:
        for beam in beams:
            try:
                lats = f[f'/{beam}/latitudes'][()]
                lons = f[f'/{beam}/longitudes'][()]
                coeffs = f[f'/{beam}/correlation_coefficients'][()]
                lags = f[f'/{beam}/best_lags'][()]
                velocs = f[f'/{beam}/velocities'][()]

                xy=np.array(pyproj.proj.Proj(epsg)(lons,lats))
                ixs = coeffs > correlation_threshold

                h0 = hax0.scatter(xy[0][ixs], xy[1][ixs], 0.25, coeffs[ixs], vmin = correlation_threshold, vmax = 1)
                h1 = hax1.scatter(xy[0][ixs], xy[1][ixs], 0.25, velocs[ixs], vmin = -700, vmax = 700,cmap='RdBu')#,cmap='RdBu')
            except:
                
                pass
            
c = plt.colorbar(h0, ax = hax0)
c.set_label('Correlation coefficient (0 -> 1)')

c = plt.colorbar(h1, ax = hax1)
c.set_label('Along-track velocity (m/yr)')

outfile = out_path + 'results_masked.png'
plt.savefig(outfile)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356200.,  183825.,  561950.]), 'origin': 'lower'}
{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356200.,  183825.,  561950.]), 'origin': 'lower'}


# Separate by ascending and descending tracks

In [50]:
# Plot MOA with correlation coefficient and velocity on top, separated by ascending and descending tracks
plt.close('all')
fig0 = plt.figure(figsize=[8,8])
hax0=fig0.add_subplot(211, aspect='equal')
MOA.show(ax=hax0,cmap='gray', clim=[14000, 17000])
hax1=fig0.add_subplot(212, aspect='equal')
MOA.show(ax=hax1,cmap='gray', clim=[14000, 17000])

hax0.set_title('Correlation Coefficient, above correlation threshold ' + str(correlation_threshold))
hax1.set_title('Best velocity, above correlation threshold ' + str(correlation_threshold))

fig1 = plt.figure(figsize=[8,8])
hax2=fig1.add_subplot(211, aspect='equal')
MOA.show(ax=hax2,cmap='gray', clim=[14000, 17000])
hax3=fig1.add_subplot(212, aspect='equal')
MOA.show(ax=hax3,cmap='gray', clim=[14000, 17000])

hax3.set_title('Correlation Coefficient, above correlation threshold ' + str(correlation_threshold))
hax3.set_title('Best velocity, above correlation threshold ' + str(correlation_threshold))

### Loop over results, load data, plot
results_files = glob.glob(out_path + '/*.hdf5')
for file in results_files:
    with h5py.File(file, 'r') as f:
        for beam in beams:
            try:
                lats = f[f'/{beam}/latitudes'][()]
                lons = f[f'/{beam}/longitudes'][()]
                coeffs = f[f'/{beam}/correlation_coefficients'][()]
                lags = f[f'/{beam}/best_lags'][()]
                velocs = f[f'/{beam}/velocities'][()]

                xy=np.array(pyproj.proj.Proj(epsg)(lons,lats))
                ixs = coeffs > correlation_threshold

                if np.median(lats[10:20] - lats[9:19]) >=0: # if latitude is increasing
                    h0 = hax0.scatter(xy[0][ixs], xy[1][ixs], 0.25, coeffs[ixs], vmin = correlation_threshold, vmax = 1)
                    h1 = hax1.scatter(xy[0][ixs], xy[1][ixs], 0.25, velocs[ixs], vmin = -700, vmax = 700,cmap='RdBu')#,cmap='RdBu')
                elif np.median(lats[10:20] - lats[9:19]) <0: # if latitude is decreasing
                    h2 = hax2.scatter(xy[0][ixs], xy[1][ixs], 0.25, coeffs[ixs], vmin = correlation_threshold, vmax = 1)
                    h3 = hax3.scatter(xy[0][ixs], xy[1][ixs], 0.25, velocs[ixs], vmin = -700, vmax = 700,cmap='RdBu')#,cmap='RdBu')
                    
            except:
                pass

fig0.colorbar(h0, ax = hax0)
fig0.colorbar(h1, ax = hax1)
fig1.colorbar(h2, ax = hax2)
fig1.colorbar(h3, ax = hax3)

fig0.suptitle('Descending tracks')
fig1.suptitle('Ascending tracks')


outfile = out_path + 'results_masked_descending.png'
fig0.savefig(outfile)
outfile = out_path + 'results_masked_ascending.png'
fig1.savefig(outfile)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356200.,  183825.,  561950.]), 'origin': 'lower'}
{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356200.,  183825.,  561950.]), 'origin': 'lower'}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356200.,  183825.,  561950.]), 'origin': 'lower'}
{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-887950., -356200.,  183825.,  561950.]), 'origin': 'lower'}


/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/UOCNT/rag110/anaconda3/envs/icesat2020/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/UOCNT/rag110/ana

In [73]:


spatial_extent = np.array([-65, -86, -55, -81])
#Rodrigo Gomez Fell computer path @ UC
path = '/mnt/user1/Antarctica/Quantarctica3/Glaciology/MEaSUREs Ice Flow Velocity/'
vel_x = 'anta_phase_map_VX.tif'
vel_y = 'anta_phase_map_VY.tif'
data_points = {}
temp = []
for file in results_files:
    with h5py.File(file, 'r') as f:
        for beam in beams:
            try:
                lats = f[f'/{beam}/latitudes'][()]
                lons = f[f'/{beam}/longitudes'][()]
                coeffs = f[f'/{beam}/correlation_coefficients'][()]
                lags = f[f'/{beam}/best_lags'][()]
                velocs = f[f'/{beam}/velocities'][()]

                xy=np.array(pyproj.proj.Proj(epsg)(lons,lats))
                ixs = coeffs > correlation_threshold
                data_points['x']=xy[0,:].reshape(lats.shape)
                data_points['y']=xy[1,:].reshape(lons.shape)
                
                temp.append(add_surface_velocity_to_is2_dict(data_points, spatial_extent, path, vel_x, vel_y ))
               
            except:
                
                pass


In [75]:


plt.scatter(velocs, temp['v_along'])

TypeError: list indices must be integers or slices, not str

#Look at the difference between temp Vdiff in temp